reference: [Fine-Tune Wav2Vec2 for English ASR with 🤗 Transformers](https://huggingface.co/blog/fine-tune-wav2vec2-english#set-up-trainer)

## 1.&nbsp;라이브러리 설치

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Sep 22 06:58:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.17.0
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
%%capture
!apt install git-lfs

## 2.&nbsp;데이터 로드

In [ ]:
from datasets import load_dataset, load_metric, Audio
zeroth_data = load_dataset("kresnik/zeroth_korean", 'clean')

Downloading:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.3G [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset zeroth_korean_asr downloaded and prepared to /root/.cache/huggingface/datasets/kresnik___zeroth_korean_asr/clean/1.0.1/f6cf96a53d5512525e3113bab8048d36ce268658d6e0c40d45f65dfa3f0bc343. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
zeroth_data

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 22263
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 457
    })
})

## 3.&nbsp;데이터 전처리

In [ ]:
zeroth_data = zeroth_data.remove_columns(['speaker_id', 'chapter_id', 'id'])

In [ ]:
zeroth_data

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 22263
    })
    test: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 457
    })
})

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(zeroth_data["train"].remove_columns(["file", "audio"]), num_examples=10)

,text
0,조 사무총장은 다른 항공사들 같으면 곤란하다 우리도 힘들다고 했을 만한 상황이었다
1,지난해 우리나라 무역 규모는 이 쩜 이 퍼센트 증가하고 수출은 이 쩜 사 퍼센트 수입은 이 쩜 공 퍼센트 늘었다
2,이화여대 총학생회와 서른 개 단과대 학과 학생회 회견을 열어 역사 교과서 국정화 전환은 하나의 역사관만을 올바르다고 강제하는 시대 역행적 시도라고 주장했다
3,원주민들은 책들과 기다란 종잇조각들에다가 모든 일들을 그것들이 일어난 해와 달과 일자를 계산한 것과 함께 기록하여 그렸다
4,국방부는 이 같은 조사내용을 십 일 발표하고 북 도발에 응당한 혹독한 대가를 치르게 할 것이라고 밝혔다
5,시계를 매일 하루 종일 찰 것입니까 아니면 특별한 경우에만 찰 것입니까
6,시는 조선소 개발에 막대한 예산이 들어가 마땅한 대책을 내놓지 못하고 있다
7,당시 합수부에는 서울중앙지검의 형사 삼 쩜 사 부 강력부 첨단범죄수사 이 부 범죄정보과 조직범죄수사과 수사 일 쩜 이 과 인터넷범죄수사센터 등이 대대적으로 참여했다
8,김 대표는 이날 인터뷰에서 자신이 당에 공식적으로 들어온 일 월 십 오 일을 수차례 언급하며 그 이전으로 돌아가면 희망이 없다고 강조했다
9,인구절벽 문제를 다룬 보도나 논문 등을 보면 부동산시장 붕괴 노동생산 가능인구 부족 소비절벽 등의 문제를 많이 거론한다


## 4.&nbsp;토크나이저

### Wav2Vec2CTCTokenizer

In [ ]:
import re

chars_to_ignore_regex = '[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [ ]:
zeroth_data = zeroth_data.map(remove_special_characters)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [ ]:
show_random_elements(zeroth_data["train"].remove_columns(["file", "audio"]), num_examples=10)

,text
0,지난 이십 칠 일 오후 서울 종로구 청운동주민센터 앞에서 성주지역 여덟 개 유림단체 회원들이 사드배치 회견을 열고 상소문을 청와대 국민소통 비서관에게 전달하고 있다
1,지난 구 월 구 일에도 제주의 음식점에서 오십 대 여주인을 마구 때린 혐의로 중국인 관광객 다섯 명이 구속되고 두 명이 불구속 입건됐다
2,바로 이것이 이천 칠 년 십 일 월 당시 청와대 회의록 기록이다
3,한편 타이즈는 현재 민병대와 후티 반군이 교전 중에 있다
4,이에 서울시는 과일주스의 천연과일 사용량을 높이고 첨가당을 적게 사용해 당 함량을 낮출 필요가 있다고 권고했다
5,노부부의 며느리는 이 층에서 뛰어내려 탈출하는 과정에서 허리 등에 경상을 입고 병원으로 옮겨져 치료를 받고 있다
6,아니면 정진용씨가 살기 위해 조현오 전 청장을 끌여들였다고 보나
7,이 돈은 자살로 처리된 군인에게만 지급하는 항목이기 때문에 무조건 지급할 수 없다는 말입니다
8,많은 사람들은 정체 구간에 들어서기도 전에 이미 스트레스를 받은 상태에 있습니다
9,동 지에서는 인식 부족과 시력 전문가의 부족을 인도가 안고 있는 문제의 주요 원인으로 꼽았다


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}


In [ ]:
vocabs = zeroth_data.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=zeroth_data.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 '가': 1,
 '각': 2,
 '간': 3,
 '갈': 4,
 '감': 5,
 '갑': 6,
 '값': 7,
 '갓': 8,
 '갔': 9,
 '강': 10,
 '갖': 11,
 '같': 12,
 '갚': 13,
 '개': 14,
 '객': 15,
 '갠': 16,
 '갤': 17,
 '갯': 18,
 '갱': 19,
 '거': 20,
 '걱': 21,
 '건': 22,
 '걷': 23,
 '걸': 24,
 '검': 25,
 '겁': 26,
 '것': 27,
 '겉': 28,
 '게': 29,
 '겐': 30,
 '겟': 31,
 '겠': 32,
 '겨': 33,
 '격': 34,
 '겪': 35,
 '견': 36,
 '결': 37,
 '겸': 38,
 '겹': 39,
 '겼': 40,
 '경': 41,
 '곁': 42,
 '계': 43,
 '고': 44,
 '곡': 45,
 '곤': 46,
 '곧': 47,
 '골': 48,
 '곰': 49,
 '곱': 50,
 '곳': 51,
 '공': 52,
 '곶': 53,
 '과': 54,
 '곽': 55,
 '관': 56,
 '괄': 57,
 '괌': 58,
 '광': 59,
 '괘': 60,
 '괭': 61,
 '괴': 62,
 '굉': 63,
 '교': 64,
 '구': 65,
 '국': 66,
 '군': 67,
 '굳': 68,
 '굴': 69,
 '굵': 70,
 '굶': 71,
 '굽': 72,
 '굿': 73,
 '궁': 74,
 '궈': 75,
 '권': 76,
 '궐': 77,
 '궤': 78,
 '귀': 79,
 '규': 80,
 '균': 81,
 '그': 82,
 '극': 83,
 '근': 84,
 '글': 85,
 '금': 86,
 '급': 87,
 '긋': 88,
 '긍': 89,
 '기': 90,
 '긴': 91,
 '길': 92,
 '김': 93,
 '깁': 94,
 '깃': 95,
 '깊': 96,
 '까': 97,
 '깎': 98,
 '깔': 99,
 '깝': 100,

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

1205

In [ ]:
import json
with open("vocab.json", 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
repo_name = "wav2vec2-large-xls-r-300m-korean-d"

In [ ]:
tokenizer.push_to_hub(repo_name)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:42: FutureWarning: Deprecated positional argument(s) used in 'create_repo': pass token='wav2vec2-large-xls-r-300m-korean-d' as keyword args. From version 0.12 passing these as positional arguments will result in an error,
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:684: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/thisisHJLee/wav2vec2-large-xls-r-300m-korean-d into local empty directory.
To https://huggingface.co/thisisHJLee/wav2vec2-large-xls-r-300m-korean-d
   c512700..f7cd8f1  main -> main

   c512700..f7cd8f1  main -> main



'https://huggingface.co/thisisHJLee/wav2vec2-large-xls-r-300m-korean-d/commit/f7cd8f1602953cb57d6f2990b732f121eed100a1'

### Wav2Vec2 Feature Extractor

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

### Preprocess Data

In [ ]:
zeroth_data["train"][0]["file"]

'/root/.cache/huggingface/datasets/downloads/extracted/3c93119fdbcba519e1529c416a7339ed19b67c18686fc5bea5eda3309e01e58e/train_data_01/003/146/146_003_0005.flac'

In [ ]:
zeroth_data["train"][0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/3c93119fdbcba519e1529c416a7339ed19b67c18686fc5bea5eda3309e01e58e/train_data_01/003/146/146_003_0005.flac',
 'array': array([ 1.2207031e-04,  1.8310547e-04,  9.1552734e-05, ...,
        -1.2207031e-04,  1.8310547e-04, -1.2207031e-04], dtype=float32),
 'sampling_rate': 16000}

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(zeroth_data["train"]))

print(zeroth_data["train"][rand_int]["text"])
ipd.Audio(data=zeroth_data["train"][rand_int]["audio"]["array"], autoplay=True, rate=16000)

한 현직 영어강사는 토익은 다른 공인영어시험에 비해 유형이 잘 알려져 있고 스터디나 수업을 꾸리기에 용이하다 


In [ ]:
rand_int = random.randint(0, len(zeroth_data["train"]))

print("Target text:",zeroth_data["train"][rand_int]["text"])
print("Input array shape:", np.asarray(zeroth_data["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", zeroth_data["train"][rand_int]["audio"]["sampling_rate"])

Target text: 사회모임에서는 별다른 신분 확인이 없는 데다 남들에게 어떻게 보이는지 무슨 말을 하는지가 중요하다는 점을 노린 것이다 
Input array shape: (139942,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [ ]:
zeroth_data = zeroth_data.map(prepare_dataset, remove_columns=zeroth_data.column_names["train"], num_proc=4)

In [ ]:
max_input_length_in_sec = 5.0
zeroth_data["train"] = zeroth_data["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

  0%|          | 0/23 [00:00<?, ?ba/s]

## 5.&nbsp;Trainer 준비

Set-up Trainer

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
cer_metric = load_metric("cer")

Downloading:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [ ]:
from transformers import Wav2Vec2ForCTC

'''
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)
'''

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-xls-r-300m were not used when initializing Wav2Vec2ForCTC: ['project_hid.weight', 'quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_hid.bias', 'project_q.weight', 'project_q.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it 

In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1712: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


In [ ]:
from transformers import TrainingArguments

'''
training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=2,
  dataloader_num_workers = 1,
  evaluation_strategy="steps",
  num_train_epochs=5,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=1e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)
'''

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=20,
  fp16=True,
  gradient_checkpointing=True, 
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
  push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=zeroth_data["train"],
    eval_dataset=zeroth_data["test"],
    tokenizer=processor.feature_extractor,
)

/content/wav2vec2-large-xls-r-300m-korean-d is already a clone of https://huggingface.co/thisisHJLee/wav2vec2-large-xls-r-300m-korean-d. Make sure you pull the latest changes with `repo.git_pull()`.
Using amp half precision backend


##6.&nbsp;모델 훈련

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1230
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3080


Step,Training Loss,Validation Loss,Cer
500,24.413500,7.774853,1.000000
1000,4.786600,5.185071,1.000000
1500,4.439600,4.857534,0.956776
2000,4.311600,4.937901,0.953204
2500,4.193100,4.716913,0.919284
3000,3.951700,4.373694,0.839628


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 457
  Batch size = 8
Saving model checkpoint to wav2vec2-large-xls-r-300m-korean-d/checkpoint-500
Configuration saved in wav2vec2-large-xls-r-300m-korean-d/checkpoint-500/config.json
Model weights saved in wav2vec2-large-xls-r-300m-korean-d/checkpoint-500/pytorch_model.bin
Feature extractor saved in wav2vec2-large-xls-r-300m-korean-d/checkpoint-500/preprocessor_config.json
Feature extractor saved in wav2vec2-large-xls-r-300m-korean-d/preprocessor_config.json
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.

TrainOutput(global_step=3080, training_loss=7.582707551237824, metrics={'train_runtime': 3480.4296, 'train_samples_per_second': 7.068, 'train_steps_per_second': 0.885, 'total_flos': 3.4784294751165676e+18, 'train_loss': 7.582707551237824, 'epoch': 20.0})

## 7.&nbsp;모델 저장

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to wav2vec2-large-xls-r-300m-korean-b
Configuration saved in wav2vec2-large-xls-r-300m-korean-b/config.json
Model weights saved in wav2vec2-large-xls-r-300m-korean-b/pytorch_model.bin
Configuration saved in wav2vec2-large-xls-r-300m-korean-b/preprocessor_config.json


Upload file pytorch_model.bin:   0%|          | 3.33k/1.18G [00:00<?, ?B/s]

Upload file runs/Sep22_02-33-20_74b8d59390c6/events.out.tfevents.1663814016.74b8d59390c6.76.4:  75%|#######4  …

Upload file runs/Sep22_02-33-20_74b8d59390c6/1663814016.420088/events.out.tfevents.1663814016.74b8d59390c6.76.…

Upload file runs/Sep22_02-33-47_74b8d59390c6/1663814043.9441848/events.out.tfevents.1663814043.74b8d59390c6.76…

...

Upload file runs/Sep22_02-33-47_74b8d59390c6/events.out.tfevents.1663814043.74b8d59390c6.76.6:  75%|#######4  …

Upload file runs/Sep22_02-08-21_74b8d59390c6/events.out.tfevents.1663812575.74b8d59390c6.76.0:  81%|########  …

Upload file training_args.bin: 100%|##########| 2.80k/2.80k [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/thisisHJLee/wav2vec2-large-xls-r-300m-korean-b
   18c2b46..60b371f  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/thisisHJLee/wav2vec2-large-xls-r-300m-korean-b
   18c2b46..60b371f  main -> main

Dropping the following result as it does not have all the necessary field:
{}
To https://huggingface.co/thisisHJLee/wav2vec2-large-xls-r-300m-korean-b
   60b371f..7909869  main -> main

   60b371f..7909869  main -> main



'https://huggingface.co/thisisHJLee/wav2vec2-large-xls-r-300m-korean-b/commit/60b371f91cb16cfbd2eb2cddde20249ba2390f97'

## 8.&nbsp;모델 평가

In [ ]:
processor = Wav2Vec2Processor.from_pretrained(repo_name)
model = Wav2Vec2ForCTC.from_pretrained(repo_name)

loading feature extractor configuration file wav2vec2-large-xls-r-300m-korean-b/preprocessor_config.json
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

Didn't find file wav2vec2-large-xls-r-300m-korean-b/added_tokens.json. We won't load it.
Didn't find file wav2vec2-large-xls-r-300m-korean-b/tokenizer.json. We won't load it.
loading file wav2vec2-large-xls-r-300m-korean-b/vocab.json
loading file wav2vec2-large-xls-r-300m-korean-b/tokenizer_config.json
loading file None
loading file wav2vec2-large-xls-r-300m-korean-b/special_tokens_map.json
loading file None
Adding <s> to the vocabulary
Adding </s> to the vocabulary
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
loading configuration file wav2vec2-large-xls-r-

In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [ ]:
results = zeroth_data["test"].map(map_to_result, remove_columns=zeroth_data["test"].column_names)

0ex [00:00, ?ex/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [ ]:
print("Test CER: {:.3f}".format(cer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test CER: 0.897


In [ ]:
show_random_elements(results)

,pred_str,text
0,그 이 있니다,업계 관계자는 가격이 오르면 지상파 삼 사는 연간 천 이백 억원 추가 수익을 올린다면서 그 부담은 이용자에게 고스란히 넘어갈 수밖에 없다고 토로했다
1,그러 했다,그는 워크아웃에 대한 확대 해석에 선을 그으면서도 사업 다각화를 위한 투자 실패로 재무구조가 악화된 것은 사실이라고 덧붙였다
2,그 이 이다,해당 공무원은 삼십 대 중반에 늦깎이로 시험에 합격했지만 일 년도 안 돼 공직자 옷을 벗었다
3,그 습니다,그는 스코틀랜드와 아일랜드를 우회해서 스페인으로 돌아가기로 결정했습니다
4,그러 이 있이요,만일 내가 그 사람과 결혼했더라면 그리스도인 집회에 혼자 다닐 뻔했어요
5,그 어 있다고 고 했다,연구진은 남편의 경우 아내와 사이가 틀어진 후에도 충분한 수면시간이 확보된 경우라면 아내와의 결혼생활에 대해 낙관적인 자세를 유지하는 것으로 나타났다고 설명했다
6,그 고 다 다 있습니다,지금 어딘가에서 냉정하게 열심히 쓰고 있다는 한강 작가의 그 다음 작품 그 작품도 기대하고 있겠습니다
7,그 이 습니다,기후가 더 좋은 그곳에서 대학교를 다니고 운동과 관련된 분야에서 일을 할 생각이었습니다
8,그 이 다 이 했다,지난해 짜장 짬뽕 라면으로 중화요리 전쟁을 벌인데 이어 올 가을 부대찌개 라면으로 다시 한 번 프리미엄 제품 마케팅 경쟁이 예고됐다
9,그 이 했다,매장 곳곳에는 땡큐 세일 입간판과 영업종료 안내문이 세워졌다


In [ ]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(zeroth_data["test"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [ ]:
# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

'그 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | | | | | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]